In [2]:
import numpy as np
import dlib
import cv2
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from colormath.color_objects import LabColor
from colormath.color_diff import delta_e_cie1976
import colorsys
import csv
import pandas as pd


In [3]:
image = cv2.imread('12.jpg')

# 읽어온 이미지 show
    # cv2.imshow("read iamge", image)
    # cv2.waitKey(0)


    # 읽어온 image 색체계 BGR -> Gray
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # gray로 바꾼 이미지 show
    # cv2.imshow("Bgr -> Gray", gray)
    # cv2.waitKey(0)

    # 면상 인식 모델 가져오기
FaceDetector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")

    # 가져온 모델에 gray 색체계로 만들 image 삽입
faces = FaceDetector(gray)

    # 얼굴 인식
for face in faces:
    # 얼굴 x1, x2 ,y1, y2 함수 통해서 가져옴
    x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()

        # 가져온 4개의 자표로 얼굴 사각형 그리기
    cv2.rectangle(image, pt1=(x1, y1), pt2=(x2, y2), color=(0, 255, 0), thickness=2)

        # 사각형으로 그린부분 잘라서 img 변수에 저장
    img = image[y1:y2, x1:x2]

    # 사각형으로 자른 얼굴 show
    # cv2.imshow("face recognization and save", img)
    # cv2.waitKey(0)

    # img 폴터에 이미지이름_1.jpg 형식으로 저장
# cv2.imwrite('./img/', '_1.jpg', img)

    # 얼굴 인식해서 잘라 저장한 img를 BGR -> Gray 로 색체계 변경
    # gray 변수에 저장
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # gray 저장한거 show
    # cv2.imshow("Face Landmark", gray)
    # cv2.waitKey(0)

    # 다시 얼굴 넣기
faces = FaceDetector(gray)
   # height, width, channel 변수에 저장
h, w, c = img.shape

# height, width, channel 크기에 따라
# 적당한 크기의 써클을 그리기 위해
# 여러 수치로 테스트 해보다가 40이 적당해서
# 40으로 나눔
# 그래서 circle size를 설정함
circle_size = (h + w) // 30

# 이제 68개의 점을 그릴건데
# # 위에서 구한 circle_size로 그릴거임
# for face in faces:
#         shape = predictor(gray, face)
#         for n in range(0, 68):
#             x = shape.part(n).x
#             y = shape.part(n).y
#             cv2.circle(img, (x, y), circle_size, (0, 0, 0), -1)

# 짠 다 그려서 한번 보자
# cv2.imshow("draw circle", img)
# cv2.waitKey(0)

In [6]:
# YCrCb 색체계의 사람 피부색 범위를 알기 때문에
# 일단 위에서 원까지 다그린 img 이미지를 YCrCb로 바꿔줌
# 그걸 face_img_ycrcb에 저장
face_img_ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
# 그래서 ycrcb 사람 피부색 범위가 (00, 133, 77) ~ (255, 173, 127)이 범위래
# 근데 첫번째 00~255은 그냥 30 ~ 255로 내가 바꿈
# 왜냐 이게 더 잘되는 듯?
# 그리고 논문에도 133 ~ 173 이랑 77 ~ 127 내용은 봤는데
# 00 ~ 255는 못본듯? 다시 봐야될 듯
# 뭐 아무튼 이렇게 최솟값 lower랑 최댓값 upper를 저장함
lower = np.array([30,133,77], dtype = np.uint8)
upper = np.array([255,173,127], dtype = np.uint8)

# 그리고 위에 face_img_ycrcb 저장한거에
# inRange 함수써서 mask 엎어버리고
skin_msk = cv2.inRange(face_img_ycrcb, lower, upper)


# 이걸 기존 이미지에 마스크 해줌
skin = cv2.bitwise_and(img, img, mask = skin_msk)

# 그리고 보면 이렇게 보일 거임
# 피부색 범위가 아닌 요소는 검은색으로 덮어 버린거
# cv2.imshow("skin_msk", skin)
# cv2.waitKey(0)

# 그리고 이제 RGB 값을 추출할거라]
# BGR -> RGB로 바꿔줌
# image = cv2.cvtColor(skin, cv2.COLOR_BGR2RGB)
cv2.imshow("skin_msk", skin)
cv2.waitKey(0)

27

In [9]:
import cv2
import numpy as np
img = skin
#converting from gbr to hsv color space
img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#skin color range for hsv color space 
HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17,170,255)) 
HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
#converting from gbr to YCbCr color space
img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
#skin color range for hsv color space 
YCrCb_mask = cv2.inRange(img_YCrCb, (30, 133, 77),(255, 173, 127)) 
YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

#merge skin detection (YCbCr and hsv)
global_mask=cv2.bitwise_and(YCrCb_mask,HSV_mask)
global_mask=cv2.medianBlur(global_mask,3)
global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((4,4), np.uint8))


HSV_result = cv2.bitwise_not(HSV_mask)
YCrCb_result = cv2.bitwise_not(YCrCb_mask)
global_result=cv2.bitwise_not(global_mask)
mask_res = cv2.bitwise_and(img, img, mask = HSV_mask)

#back_mask skin color
img1 = np.zeros((480, 640, 3), dtype=np.uint8) 
h,w,c = img.shape
skin_tone = np.full((h,w,c),(203,218,243),dtype=np.uint8)

mask_res2 = cv2.bitwise_and(skin_tone, skin_tone, mask = HSV_mask)

mask_res3 = cv2.bitwise_and(mask_res2, img, mask = HSV_result)

mask_res4 = cv2.bitwise_and(mask_res3, img, mask = HSV_result)
# 241,218,203
#show results
cv2.imshow("1_HSV.jpg",YCrCb_result)
# cv2.imshow("2_YCbCr.jpg",YCrCb_result)
# cv2.imshow("3_global_result.jpg",global_result) 
cv2.imshow("sa.jpg",skin_tone)
# cv2.imshow("Image.jpg",img)
cv2.imshow("q1.jpg",mask_res2)
cv2.imshow("q2.jpg",mask_res3)
cv2.imshow("q3.jpg",mask_res4)

print(h,w,c)
# cv2.imshow('img',img)
# cv2.imwrite("1_HSV.jpg",HSV_result)
# cv2.imwrite("3_global_result.jpg",global_result)
cv2.waitKey(0)
cv2.destroyAllWindows()  

258 258 3


KeyboardInterrupt: 

: 